###Cloning the llama.cpp repo and installing dependencies

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git
!cd llama.cpp &&  pip install -r requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 42763, done.
remote: Counting objects: 100% (8049/8049), done.
remote: Compressing objects: 100% (416/416), done.
remote: Total 42763 (delta 7855), reused 7635 (delta 7633), pack-reused 34714 (from 2)
Receiving objects: 100% (42763/42763), 75.64 MiB | 20.53 MiB/s, done.
Resolving deltas: 100% (31389/31389), done.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu124
    Uninstalling torch-2.5.1+cu124:
      Successfully uninstalled torch-2.5.1+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

###Building llama.cpp locally

In [ ]:
!cd llama.cpp && LLAMA_CUDA=1 && pwd && cmake -B build && cmake --build build --config Release -j$(nproc)

/content/llama.cpp
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP: TRUE 

###Downloading deepseek model from hugging face official repo

In [ ]:
from huggingface_hub import snapshot_download

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
original_model= "./original_model/"
snapshot_download(repo_id=model_id, local_dir=original_model,local_dir_use_symlinks=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:832: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.6k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

...

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

'/content/original_model'

In [ ]:
!./llama.cpp/build/bin/llama-cli --version

version: 4607 (aa6fb132)
built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu


###Converting tensors from hf to .gguf format for quantization

In [ ]:
!mkdir ./quantized_model/
!python llama.cpp/convert_hf_to_gguf.py ./original_model --outtype f16 --outfile ./quantized_model/FP16.gguf

INFO:hf-to-gguf:Loading model: original_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-000002.safetensors'
INFO:hf-to-gguf:token_embd.weight,         torch.bfloat16 --> F16, shape = {3584, 152064}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.bfloat16 --> F32, shape = {3584}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.bfloat16 --> F16, shape = {18944, 3584}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torch.bfloat16 --> F16, shape = {3584, 18944}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.bfloat16 --> F16, shape = {3584, 18944}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.bfloat16 --> F32, shape = {3584}
INFO:hf-to-gguf:blk.0.attn_k.bias,         torch.bfloat16 --> F32, shape = {512}
INFO:hf-to-gguf:blk.0.attn_k.weight,       torch.bfloat16 --> F16, shape = {3584, 512}
I

###Quantising the model

In [ ]:
import os

methods = ["q4_k_m"]
quantized_path = "./quantized_model/"

for m in methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [ ]:
!./llama.cpp/build/bin/llama-quantize --pure /content/quantized_model/FP16.gguf /content/quantized_model/q4_k_m.gguf Q4_K_M

main: build = 4607 (aa6fb132)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/quantized_model/FP16.gguf' to '/content/quantized_model/q4_k_m.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 26 key-value pairs and 339 tensors from /content/quantized_model/FP16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 7.6B
llama_model_loader: - kv   4:                            general.license str              = mit
llama_model_loader: - kv   5:                  

###Checking the quantised model

In [ ]:
! ./llama.cpp/build/bin/llama-cli -m /content/quantized_model/q4_k_m.gguf  -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/reason-act.txt

build: 4607 (aa6fb132) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 26 key-value pairs and 339 tensors from /content/quantized_model/q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 7.6B
llama_model_loader: - kv   4:                            general.license str              = mit
llama_model_loader: - kv   5:                          qwen2.block_count u32              = 2

###Uploading to hugging face

In [ ]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [ ]:
from huggingface_hub import login, upload_file


login(os.environ["HF_TOKEN"])


model_path = "./quantized_model/q4_k_m.gguf"
repo_id = "Rewatiramans/DeepSeek-R1-Distill-Qwen-7B-Q4-GGUF"


upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q4_K_M.gguf",
    repo_id=repo_id,
    repo_type="model",
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


q4_k_m.gguf:   0%|          | 0.00/4.29G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rewatiramans/DeepSeek-R1-Distill-Qwen-7B-Q4-GGUF/commit/328c97b10ab1537b56ec3f60b37cfe8cb52c5eb5', commit_message='Upload Q4_K_M.gguf with huggingface_hub', commit_description='', oid='328c97b10ab1537b56ec3f60b37cfe8cb52c5eb5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rewatiramans/DeepSeek-R1-Distill-Qwen-7B-Q4-GGUF', endpoint='https://huggingface.co', repo_type='model', repo_id='Rewatiramans/DeepSeek-R1-Distill-Qwen-7B-Q4-GGUF'), pr_revision=None, pr_num=None)